In [2]:
# Please install OpenAI SDK first: `pip3 install openai`
from openai import OpenAI

my_api_key = "sk-ec35ec1c158049819f9bf2db1af5784e"
client = OpenAI(api_key=my_api_key, base_url="https://api.deepseek.com")

In [ ]:
import pandas as pd
# 读取operators.xlsx文件
df_operators = pd.read_excel('operators.xlsx')
df_operators.head()

def convert_operator_to_text(df):
    text_parts = []
    for index, row in df_operators.iterrows():
        operator = row['Operator']
        description = row['Description']
        example = row['Example']
        part = f"操作符: {operator}\n描述: {description}\n例子: {example}\n{'='*30}\n"
        text_parts.append(part)
    return ''.join(text_parts)

# 转换为文本
operators = convert_operator_to_text(df_operators)
# print(operators)

In [15]:

system_prompt = """
你是一个金融量化领域专家，你首先需要完成以下工作：
1. 理解发送给你的operators的含义及使用方法；
2. 理解发送给你的金融数据的字段含义及使用方法；

在完成以上工作后，你需要完成以下工作：
1. 根据operators和金融数据，构造alpha表达式；
2. 对alpha表达式进行优化，使alpha表达式的回测结果最好；
3. 输出优化后的alpha表达式。
"""


system_prompt = """
你是一个金融量化领域专家，你首先需要完成以下工作：
1. 理解发送给你的operators的含义及使用方法；
2. 理解发送给你的金融数据的字段含义及使用方法；

在完成以上工作后，你需要完成以下工作：
1. 根据operators和金融数据，构造一些alpha表达式；
"""

user_prompt = """
根据以下数据，构造alpha表达式：
operators: {operators}
financial_data: {"volume,close,open,high,low"}
"""



In [16]:
response = client.chat.completions.create(
    model="deepseek-chat",
    messages=[
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": user_prompt},
    ],
    stream=False
)

print(response.choices[0].message.content)

# Alpha表达式构造

作为金融量化专家，我将基于提供的operators和金融数据字段(volume, close, open, high, low)构造一些有效的alpha表达式。

## 理解数据字段
- volume: 成交量
- close: 收盘价
- open: 开盘价
- high: 最高价
- low: 最低价

## 基础Alpha表达式

1. **价格动量因子**
   ```
   (close - delay(close, 5)) / delay(close, 5)
   ```
   计算5日价格变化率，反映短期动量

2. **成交量加权价格**
   ```
   sma(volume * close, 10) / sma(volume, 10)
   ```
   10日成交量加权平均价格

3. **波动率因子**
   ```
   (high - low) / close
   ```
   当日价格波动幅度

4. **量价背离因子**
   ```
   correlation(close, volume, 5)
   ```
   5日收盘价与成交量的相关性

5. **突破因子**
   ```
   (close > sma(high, 5)) ? 1 : (close < sma(low, 5)) ? -1 : 0
   ```
   判断价格是否突破5日高低点区间

## 复合Alpha表达式

6. **量价协同动量**
   ```
   (close - delay(close, 5)) / delay(close, 5) * log(volume / sma(volume, 5))
   ```
   结合价格动量和成交量变化

7. **日内强度指标**
   ```
   (2*close - low - high) / (high - low) * volume
   ```
   衡量日内价格走势强度并乘以成交量

8. **自适应波动因子**
   ```
   ts_rank(high - low, 20) / ts_rank(close, 20)
   ```
   20日波动幅度与价格水平的相对排名

9. **量价趋势确认**
   ```
   sign(close - open